# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

* Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
    * excel and vensim are only supported on windows
    * vensim requires 32 bit python, and a 7.1!! version of vensim DSS
    * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get it from [NetLogo 6.0](https://ccl.northwestern.edu/netlogo/download.shtml) 
    * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
    * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).
    

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes example scripts for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel). 

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function. 

In [2]:
import sys
!{sys.executable} -m pip install pysd

  Created wheel for parsimonious: filename=parsimonious-0.8.1-cp37-none-any.whl size=42715 sha256=c0df6c712f796a1fffcb2738b8f3ddb13328fa6049613eecefd93caa4db14551
  Stored in directory: C:\Users\pratheepkumaar\AppData\Local\pip\Cache\wheels\b7\8d\e7\a0e74217da5caeb3c1c7689639b6d28ddbf9985b840bc96a9a
  Created wheel for pydoe: filename=pyDOE-0.3.8-cp37-none-any.whl size=18183 sha256=aa5855937c9ac058368c5c336abf837b6d61ea142d0bffd11f3f517832f3423b
  Stored in directory: C:\Users\pratheepkumaar\AppData\Local\pip\Cache\wheels\7c\c8\58\a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built parsimonious pydoe


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments,
                           ema_logging)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis.plotting import lines, Density


# .. codeauthor::  jhkwakkel


def PredPrey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, final_time=365, reps=1):

    #Initial values
    predators, prey, sim_time = [np.zeros((reps, int(final_time/dt)+1)) for _ in range(3)]
    
    for r in range(reps):
        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

        #Calculate the time series
        for t in range(0, sim_time.shape[1]-1):

            dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
            dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

            prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
            predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
            sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

C:\Users\pratheepkumaar\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:17: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
C:\Users\pratheepkumaar\Anaconda3\lib\site-packages\ema_workbench\analysis\prim.py:31: ImportWarning: altair based interactive inspection not available
  "inspection not available"), ImportWarning)


In [12]:
from __future__ import (division, unicode_literals, absolute_import,
                        print_function)

from ema_workbench import (TimeSeriesOutcome, perform_experiments,
                           RealParameter, ema_logging)

from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments)

from ema_workbench.connectors.pysd_connector import PysdModel


# .. codeauthor::  jhkwakkel

if __name__ == "__main__":
    ema_logging.log_to_stderr(ema_logging.INFO)
    wd = r"./model/PredPrey.mdl"
    pysd_model = PysdModel("pysd",mdl_file=r"./model/PredPrey.mdl")

    uncertainty = [RealParameter('prey_birth_rate', 0.015, 0.035),
                RealParameter('predation_rate', 0.0005, 0.003),
                RealParameter('predator_efficiency', 0.001, 0.004),
                RealParameter('predator_loss_rate', 0.04, 0.08)]

    outcome = [TimeSeriesOutcome('TIME'),
            TimeSeriesOutcome('predators'),
            TimeSeriesOutcome('prey')]

    pysd_model.uncertainties = uncertainty
    pysd_model.outcome = outcome

    with MultiprocessingEvaluator(pysd_model) as evaluator:
        results = evaluator.perform_experiments(50)

    

   


[MainProcess/INFO] pool started
[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 15 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 25 cases completed
[MainProcess/INFO] 30 cases completed
[MainProcess/INFO] 35 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 45 cases completed
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [1]:
from __future__ import unicode_literals, absolute_import

from ema_workbench.connectors.netlogo import NetLogoModel

from ema_workbench import (RealParameter, ema_logging,
                           TimeSeriesOutcome, MultiprocessingEvaluator)

from ema_workbench import SequentialEvaluator

# Created on 20 mrt. 2013
#
# .. codeauthor::  jhkwakkel


if __name__ == '__main__':
    # turn on logging
    ema_logging.log_to_stderr(ema_logging.INFO)

    model = NetLogoModel("netlogo",wd=r"./model/",model_file="PredPrey.nlogo")
    model.run_length = 1
    model.replications = 1

    model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                RealParameter('predation_rate', 0.0005, 0.003),
                RealParameter('predator_efficiency', 0.001, 0.004),
                RealParameter('predator_loss_rate', 0.04, 0.08)]

    model.outcomes =  [TimeSeriesOutcome('TIME'),
            TimeSeriesOutcome('predators'),
            TimeSeriesOutcome('prey')]

    # perform experiments
    n = 50

    with MultiprocessingEvaluator(model) as evaluator:
        results = evaluator.perform_experiments(n)

C:\Users\pratheepkumaar\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:17: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments


In [4]:
from __future__ import (division, unicode_literals, absolute_import,
                        print_function)

from ema_workbench import (TimeSeriesOutcome, perform_experiments,
                           RealParameter, ema_logging)


from ema_workbench.connectors.excel import ExcelModel

from ema_workbench import SequentialEvaluator


# .. codeauthor::  jhkwakkel

if __name__ == "__main__":
    ema_logging.log_to_stderr(ema_logging.INFO)
    wd = r"./model/PredPrey.mdl"
    excel_model = ExcelModel('excel', wd='./model/', model_file='PredPrey.xlsx')


    uncertainty = [RealParameter('prey_birth_rate', 0.015, 0.035),
                RealParameter('predation_rate', 0.0005, 0.003),
                RealParameter('predator_efficiency', 0.001, 0.004),
                RealParameter('predator_loss_rate', 0.04, 0.08)]

    outcome = [TimeSeriesOutcome('TIME'),
            TimeSeriesOutcome('predators'),
            TimeSeriesOutcome('prey')]

    excel_model.default_sheet = "Sheet1"

    excel_model.uncertainties = uncertainty
    excel_model.outcome = outcome

    with SequentialEvaluator(excel_model) as evaluator:
        results = evaluator.perform_experiments(50)

[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 15 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 25 cases completed
[MainProcess/INFO] 30 cases completed
[MainProcess/INFO] 35 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 45 cases completed
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] experiments finished


In [6]:
from __future__ import (division, unicode_literals, absolute_import,
                        print_function)

from ema_workbench import (TimeSeriesOutcome, perform_experiments,
                           RealParameter, ema_logging)

from PredPrey import PredPrey

from ema_workbench import SequentialEvaluator,Model


# .. codeauthor::  jhkwakkel

if __name__ == "__main__":
    ema_logging.log_to_stderr(ema_logging.INFO)
  
    python_model = Model('python', function=PredPrey)


    uncertainty = [RealParameter('prey_birth_rate', 0.015, 0.035),
                RealParameter('predation_rate', 0.0005, 0.003),
                RealParameter('predator_efficiency', 0.001, 0.004),
                RealParameter('predator_loss_rate', 0.04, 0.08)]

    outcome = [TimeSeriesOutcome('TIME'),
            TimeSeriesOutcome('predators'),
            TimeSeriesOutcome('prey')]

    python_model.uncertainties = uncertainty
    python_model.outcome = outcome

    with SequentialEvaluator(python_model) as evaluator:
        results = evaluator.perform_experiments(50)

[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 15 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 25 cases completed
[MainProcess/INFO] 30 cases completed
[MainProcess/INFO] 35 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 45 cases completed
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] experiments finished
